# Шаблон для проекта по A/B - тестам.

### `chat_id ` из Телеграма:

In [26]:
chat_id = 460109099

------------------

##  <ins>Задание 1.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project1_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

Так как данные по среднему доходу ждать нет времени, можно отслеживать только сам факт успешности звонков, т.е. конверсию из дозвона в продажу. Это и будет целевой метрикой.

In [2]:
import pandas as pd
data1 = pd.read_csv('hist_telesales.csv').drop(columns='ID')

In [12]:
data1.head(20)

,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,1,0,90,0,-90
1,0,0,5,0,-5
2,0,0,68,0,-68
3,1,0,22,0,-22
4,1,0,22,0,-22
5,1,1,367,2397,2030
6,1,1,49,1213,1164
7,0,0,26,0,-26
8,0,0,35,0,-35
9,0,0,61,0,-61


In [16]:
conv = data1[data1["Флаг дозвона"]==1]["Флаг продажи"].mean()
print('конверсия из дозвона в продажу:', conv)
uplift = 0.08
MDE = conv * uplift
print('\nMDE:', MDE)

конверсия из дозвона в продажу: 0.503719275549806

MDE: 0.04029754204398448


### 2. Альтернатива в критерии.

Выбираем двухстороннюю альтернативу с гипотезой о равенстве средних. Если по критерию средние не совпадут, посмотрим на конверсии и сделаем выводы.

### 3.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

In [19]:
from scipy.stats import norm

k = 0.5 # разбитие потока равновероятно
alpha = 0.05 # уровень значимости
beta = 0.8 # уверенность в различии изменений целевой метрики

Узнаем размер выборки теста по формуле (для двухсторонней альтернативы):

In [25]:
n = conv*(1-conv) * ( (norm.ppf(alpha/2) - norm.ppf(beta))**2 )/( 0.5*MDE**2 )
print('\nРазмер выборки:', int(n))


Размер выборки: 2416


### 3.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project1_sample {размер выборки}` и вписать <ins>код выборки</ins>:

In [24]:
38277

38277

In [42]:
control1 = pd.read_csv("Контроль.csv").drop(columns="ID")
test1 = pd.read_csv("Тест.csv").drop(columns="ID")

### 4. Принятие решения. Расписать подробно с аргументами.

In [30]:
from statsmodels.stats.proportion import proportions_ztest

In [43]:
count_control1 = control1[control1["Флаг дозвона"]==1]["Флаг продажи"].sum()
count_test1 = test1[test1["Флаг дозвона"]==1]["Флаг продажи"].sum()
nobs_control1 = len(control1[control1["Флаг дозвона"]==1])
nobs_test1 = len(test1[test1["Флаг дозвона"]==1])

_, p_value = proportions_ztest([count_control1, count_test1], [nobs_control1, nobs_test1], alternative = "two-sided")
print('p_value =', p_value)

p_value = 1.074048634319037e-05


Критерий отвергает гипотезу, посмотрим среднее по данным.

In [44]:
print('Контроль:', control1[control1["Флаг дозвона"]==1]["Флаг продажи"].mean())
print ('Тест:    ', test1[test1["Флаг дозвона"]==1]["Флаг продажи"].mean())

Контроль: 0.5249146757679181
Тест:     0.44360902255639095


Как видно, конверсия с жёстким скриптом больше, поэтому стоит оставить все как было.

------------------

##  <ins>Задание 2.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project2_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

Необходимо найти доходность услуги, значит, будем смотреть конверсию из утилизации счета в NPV. PV кредитной карты и PV услуги не так показательны, так как себестоимость вместе с затратами может не оправдывать стоимость тарифа.

Целевая метрика - конверсия из утилизации счета в NPV.

In [8]:
data2 = pd.read_csv('hist_credit_card.csv').drop(columns='ID')

In [9]:
data2.head(20)

,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,19,21620.835463,0.138061,0,102,0,0,-102
1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,50,23989.526947,0.098793,0,16,0,0,-16
3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,24,21291.521612,0.036909,1,1048,6594,2213,7759
5,19,22957.947147,0.021514,1,110,33316,1692,34898
6,27,22217.477344,0.319460,0,57,0,0,-57
7,46,29584.430233,0.324886,1,371,5859,2529,8017
8,27,21730.692059,0.254344,1,712,9981,3758,13027
9,41,34909.175512,0.017871,1,131,8794,5315,13978


### 2. Альтернатива в критерии.

### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

### 4.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

### 4.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project2_sample {размер выборки}` и вписать <ins>код выборки</ins>:

### 5. Проверка на однородность, применение критерия. Принятие решения. Расписать подробно с аргументами.